# Importing Data from Youtube using API Calls

- I want to extract data like Video ID , Video Title, and each Videos statictics like no of comments , likes, views.
- I Created an API key from the Google Cloud Youtuble Video Data API for me to Extract this data using API
- I Took T-Series as my Channel to extract and got the Channel ID
- I then did API calls to extract all the data into a Data Frame for me to Store it.

### Importing Libraries  

In [1]:
import requests
import pandas as pd
import time

### Assigning the API Key and Channel ID
- Here I hid my API Key
- I took Channel Id of the Most subscribed channel in Youtube : T-Series

In [2]:
API_KEY = "xxxx"
CHANNEL_ID = "UCq-Fj5jknLsUf-MWSy4_brA"

### Getting the Video Stats with Second API Call
- Here I commented the Dislike count as youtube no longer has dislike count and throws an error
- Here we get count of Views,likes and comments of each video in the channel with this call.

In [6]:
def get_video_stats(video_id):
    url_video_stats = "https://www.googleapis.com/youtube/v3/videos?id="+video_id+"&part=statistics&key="+API_KEY
    response_video_stats = requests.get(url_video_stats).json() 
    view_count = response_video_stats['items'][0]['statistics']['viewCount']
    like_count = response_video_stats['items'][0]['statistics']['likeCount']
    #dislike_count = response_video_stats['items'][0]['statistics']['dislikeCount']
    comment_count = response_video_stats['items'][0]['statistics']['commentCount']
    
    return video_id,video_title,upload_date,view_count,like_count,comment_count

### First API call and Storing in the Dataframe 
- The first API call is made here and the details like Video Id, Video title, When it is uploaded are extracted here
- The sleep function is used here to give API a time gap inorder for it to not immediately start the extraction process
- Page Token helps is extracting data from other pages as well.
- An "If statement" is added to extract only if its a youtube video and to ensure it does'nt grab anything else in the       data.
- I have added Error handling to make sure to only return when api calls are successfull and stop when it reaches the page   end.
- It finally returns the DataFrame with all the Details of the Videos in the Channel

In [7]:
def get_videos(df):  
    pageToken=""
    while 1:
        url = "https://www.googleapis.com/youtube/v3/search?key="+API_KEY+"&channelId="+CHANNEL_ID+"&part=snippet,id&order=date&maxResults=10000"+pageToken
        response = requests.get(url).json()
        time.sleep(5)
    
        for video in response['items']:
            if video['id']['kind'] == "youtube#video":
                video_id = video['id']['videoId']
                video_title = video['snippet']['title']
                video_title = str(video_title).replace("&amp;","")
                upload_date = video['snippet']['publishedAt']
                upload_date = str(upload_date).split("T")[0]

                get_video_stats(video_id)
                #Saving all the Data into a DataFrame
                df = df.append({'video_id':video_id,'video_title':video_title,'upload_date':upload_date,
                                'view_count':view_count,'like_count':like_count,                         
                                'comment_count':comment_count},ignore_index=True)
        try:
            if response.status_code != 200:
                break
            elif response['nextPageToken'] != None: #if none,it reached the last page and it breaks out of the loop 
                pageToken = "pageToken=" + response['nextPageToken']
      
        except :
            break   
            
    return df

### Dataframe Creation
- Here we create the dataframe with the Column Titles of the information we are about to extract
- We,then pass the dataframe to the get_videos Function to start the API Calls

In [8]:
df = pd.DataFrame(columns=["video_id","video_title","upload_date","view_count","like_count","comment_count"])

df = get_videos(df)

## View of the DataFrame 

In [9]:
df

,video_id,video_title,upload_date,view_count,like_count,comment_count
0,Ik8htcIQfh0,#Ganpati2022 🙏🏻 #GanpatiStatus 🐘 Mourya Re |Mo...,2022-08-31,11307164,555516,14349
1,sGA0npLlm-0,Jubin Nautiyal: Barsaat Ho Jaaye (Lyrical) | L...,2022-08-31,11307164,555516,14349
2,v1CfnpM9IiQ,"Jaya Kishori, Jubin Nautiyal : Mere Kanha (Lyr...",2022-08-31,11307164,555516,14349
3,R8Lhy-qNniA,#SachetParampara 👩‍❤️‍👨 #LoveSong ❣️ Tere Vich...,2022-08-31,11307164,555516,14349
4,1olfC1m34S8,#TujheBhulaDiya 😭#RanbirKapoor 👨‍🎤 #SadSongSta...,2022-08-30,11307164,555516,14349
5,dOUg-jMjN8U,MASTI TAKITA DHUM (Full Vudio) Shabaash Mithu ...,2022-08-30,11307164,555516,14349
6,01sqAN-MTus,Mere Dil Gaaye Ja (Zooby Zooby) Dhokha | R. Ma...,2022-08-30,11307164,555516,14349
7,Fd19_8DpOGg,Sabri Brothers: Teri Jawani Badi Mast Mast Hai...,2022-08-29,11307164,555516,14349
8,GxFXpkoPjzY,Pihu Pihu (Audio) : Token - The Treasure | Tar...,2022-08-29,11307164,555516,14349
9,ODUNnVFBZvw,Mere Dil Gaaye Ja (Zooby Zooby) SONG TEASER | ...,2022-08-29,11307164,555516,14349


# Transfering DF to Cloud Databases

### Installing and Importing the Libraries Required for Connecting with Cloud DataBase

In [25]:
!pip install psycopg2

In [15]:
import psycopg2 as ps

### Establishing connection with database 

In [14]:
def connect_to_db(host, db, port, username, password):
    try:
        conn = ps.connect(host=host, database=db, user=username, password=password, port=port)

    except ps.OperationalError as e:
        raise e
    else:
        print('Connected!')
        return conn

### Storing Database Details into Variables
- I hid my all my details here.
- I called the function to connect the database
- I also set autocommit to update and save the databases after each running query.
- I created a cursor object to execute our queries to database

In [16]:
#Connecting to db
host = 'database-1.xxxxxxxx.ap-south-1.rds.amazonaws.com'
db = 'xxx'
port = 5432
username = 'xxxx'
password = 'xxxx' 

conn = connect_to_db(host, db,port, username, password)
conn.autocommit = True
cur = conn.cursor()

Connected!


### Displayed First Two Rows to create a DB table

In [14]:
df.head(2)

,video_id,video_title,upload_date,view_count,like_count,comment_count
0,sGA0npLlm-0,Jubin Nautiyal: Barsaat Ho Jaaye (Lyrical) | L...,2022-08-31,14168292,330148,8888
1,v1CfnpM9IiQ,"Jaya Kishori, Jubin Nautiyal : Mere Kanha (Lyr...",2022-08-31,14168292,330148,8888


## Create a DataBase Table 
- I created a table in database to store all the values we extracted from API into Datafame
- I added error handling to ensure to there is no overriden of data and the queries executed are commited into the Database

In [15]:
#creating a table using SQL Query

def table(cur):
    create_table = ("""Create Table if not exists 
                Video_Data(
                    Video_ID varchar(255) Primary Key,
                    Video_title varchar(255) not null,
                    Upload_date date not null default current_date,
                    view_count integer not null,
                    like_count integer not null,
                    comment_count integer not null)""")
    try:
        cur.execute(create_table)
        conn.commit()
    except:
        conn.rollback()


### Calling Table Creation Fuction to Create a Table. 

In [52]:
table(cur)

### Function to Handle the Database Updates from the Dataframe and API
- The basic functionality I tried to do here is to update the DB table whenever there is any videos added in the channel.
- I also want to the exsisting videos to have an upto date count on the no of likes,views and comments.
- Explaination about each function: 
- 1. This function is to insert new rows of data into the database.
- 2. This function is to update the exsisting video details whenever any new changes are brought in from the API
- 3. This function is to check the dataframe if the video already exists or not if it does then skips the video to append again.
- 4. This function is to append the new values/rows into the database which are not in it yet
- 5. This function is to tie all the functions together with the logic mentioned above. A temp table is storing the new        video details from the api so that I dont need to take the whole dataframe and cause a lag on the daatbase query and      can improve the system performance

In [17]:
#1
def insert_into_table(curr, video_id, video_title, upload_date, view_count, like_count,comment_count):
    insert_into_video_data = ("""INSERT INTO Video_Data (video_id, video_title, upload_date,
                                 view_count, like_count,comment_count)
                                 VALUES(%s,%s,%s,%s,%s,%s);""")
    
    inserts_rows = (video_id, video_title, upload_date, view_count, like_count, comment_count)
    
    try:
        cur.execute(insert_into_video_data, inserts_rows)
        
    except (Exception, ps.DatabaseError) as error:
        print(error)

        
#2
def update_row(cur, video_id, video_title, view_count, like_count,comment_count):
    query = ("""UPDATE Video_Data
            SET video_title = %s,
                view_count = %s,
                like_count = %s,
                comment_count = %s
            WHERE video_id = %s;""")
    row_to_update = (video_title, view_count, like_count,comment_count, video_id)    
    try:
        cur.execute(query, row_to_update)
        
    except (Exception, ps.DatabaseError) as error:
        print(error)

#3
def check_if_video_exists(cur, video_id): 
    query = ("""SELECT video_id FROM Video_Data WHERE video_id = %s""")
    try:
        cur.execute(query, (video_id,))
        
    except (Exception, ps.DatabaseError) as error:
        print(error)        
        
    return cur.fetchone() is not None        

#4
def append_to_db(cur,df):
    for i, row in df.iterrows():
        insert_into_table(cur, row['video_id'], row['video_title'], row['upload_date'], row['view_count'], row['like_count'], row['comment_count'])

#5        
def update_db(cur,df):
    df_temp = pd.DataFrame(columns=['video_id', 'video_title', 'upload_date', 'view_count','like_count', 'comment_count'])
    
    for i, row in df.iterrows():
        if check_if_video_exists(cur, row['video_id']):
            update_row(cur,row['video_id'],row['video_title'],row['view_count'],row['like_count'],row['comment_count'])
        else:
            df_temp = df_temp.append(row)
    
    return df_temp

### Calling Update function.
- Here we store it into a variable as its returning a temporary datafram. 
- The use of temp table is explained above

In [18]:
new_rows = update_db(cur,df)

- Here you can see the temp table data

In [20]:
new_rows

,video_id,video_title,upload_date,view_count,like_count,comment_count
0,Ik8htcIQfh0,#Ganpati2022 🙏🏻 #GanpatiStatus 🐘 Mourya Re |Mo...,2022-08-31,11307164,555516,14349
47,O-Rnan8IOZo,Jubin Nautiyal: Barsaat Ho Jaaye | Love Song 2...,2022-08-17,11307164,555516,14349
48,bKi3ElqWhBY,"Jaya Kishori, Jubin Nautiyal : Mere Kanha | Ne...",2022-08-16,11307164,555516,14349


### Calling append function.

In [19]:
append_to_db(cur,new_rows)